# Stress and Achievement Prediction Plus Prescription

The goal of this notebook is twofold: with a given user’s inputs predict what their level of achievement and daily stress is, and then make recommendations on what they can change to improve their level of stress and achievement. Model building and training were handled with Keras/TensorFlow in a prior notebook (will add GitHub link here later, ) so that this notebook could simply load the model and perform inference for a given user in minimal time.

When this notebook is run it will use user inputs to create a dataframe, the first row of the dataframe (index 'Actual') will be the user’s inputs, then it generates additional dataframe rows keeping all the user inputs the same except incrementing/decrementing one element by one (the index label states the change). This way a user can observe how a small lifestyle change could impact their life.


This model does not use the features 'BMI_RANGE', 'SUFFICIENT_INCOME', and 'GENDER', these turned out to be poor predictors (surprisingly). for a more detailed EDA, you can view a previous notebook of mine: https://www.kaggle.com/schorsi/stress-and-achievement-ml-totw

Disclaimers: Correlation and causation are not the same, the recommendations this notebook generates may not have the expected impact. While this model is able to accurately approximate the level of daily stress, it is less precise with achievement. Achievement most likely has some confounding variables involved, so take these recommendations with a grain of salt. My recommendation would be to look at the predictions for achievement under 'Actual' and if they accurately reflect your current state then I would imagine the predictions given slight changes would also stay similarly accurate.

> Version: 1.0  code complete, some additional links and documentation will be added later. The next steps include adding a GitHub page with the model and all relevant notebooks then creating a similar notebook on Google Colab so that it's accessible to a wider audience. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
# The ordered list of columns used in inference
x_columns = [
    'FRUITS_VEGGIES',
    'PLACES_VISITED',
    'CORE_CIRCLE',
    'SUPPORTING_OTHERS',
    'SOCIAL_NETWORK',
    'DONATION',
    'TODO_COMPLETED',
    'FLOW',
    'DAILY_STEPS',
    'LIVE_VISION',
    'SLEEP_HOURS',
    'LOST_VACATION',
    'DAILY_SHOUTING',
    'PERSONAL_AWARDS',
    'TIME_FOR_PASSION',
    'DAILY_MEDITATION',
    'AGE'
]

# Dictionary containing the column description for assisting the user understanding the column values
wellbeing_dict = {
     'FRUITS_VEGGIES' : '[Between 1 and 5] HOW MANY FRUITS OR VEGETABLES DO YOU EAT EVERYDAY? In a typical day, averaging workdays and weekends.',
     'DAILY_STRESS' : '[Between 1 and 10] HOW MUCH STRESS DO YOU TYPICALLY EXPERIENCE EVERYDAY? At work or at home, due to the environment (noise, pollution, insecurity...), your co-workers or boss, or because of tragic events such as divorce, job loss, serious illness, loss of family or friends,... In average over 12 months.',
     'PLACES_VISITED' : '[Between 0 and 10] HOW MANY NEW PLACES DO YOU VISIT? Over a period of 12 months. Include new states, new cities as well as museum, places of interest and parks in your neighborhood.',
     'CORE_CIRCLE' : '[Between 0 and 10] HOW MANY PEOPLE ARE VERY CLOSE TO YOU? i.e. close family and friends ready to provide you with a long-term unconditional support.',
     'SUPPORTING_OTHERS' : '[Between 0 and 10] HOW MANY PEOPLE DO YOU HELP ACHIEVE A BETTER LIFE? A reflection of your altruism or selflessness  e.g.: caring for your family, actively supporting a friend, mentoring, coaching, developing or promoting a co-worker, ... Over a period of 12 months.',
     'SOCIAL_NETWORK' : '[Between 0 and 10] WITH HOW MANY PEOPLE DO YOU INTERACT WITH DURING A TYPICAL DAY? True interactions and dialogues at home, at work, at the gym, ... Average of workdays and weekends',
     'ACHIEVEMENT' : '[Between 0 and 10] HOW MANY REMARKABLE ACHIEVEMENTS ARE YOU PROUD OF? Over the last 12 months, personal achievements known to your family, close friends or co-workers such as: running a marathon or important race, birth, successful kids, new house or major renovation, major success at work, opening a new business, ...',
     'DONATION' : '[Between 0 and 5] HOW MANY TIMES DO YOU DONATE YOUR TIME OR MONEY TO GOOD CAUSES? Over a period of 12 months. Include financial donation, your time contribution, fundraising, volunteering, serving your country and the poor, ...',
     'BMI_RANGE' : '[1 if below 25, else 2] WHAT IS YOUR BODY MASS INDEX (BMI) RANGE? Your body mass in kg divided by the square of your height in meters ► Check the online BMI calculator such as www.cdc.gov/healthyweight/assessing/bmi/index.html. ► For instance, an adult of 6 feet and 184 pounds has a BMI of 25',
     'TODO_COMPLETED' : '[Between 1 and 10] HOW WELL DO YOU COMPLETE YOUR WEEKLY TO-DO LISTS? Include your weekly goals, work- and personal-related tasks. On a scale of 0 = not at all to 10 = very well.',
     'FLOW' : '[Between 0 and 10] IN A TYPICAL DAY, HOW MANY HOURS DO YOU EXPERIENCE "FLOW"? `Flow` is defined as the mental state, in which you are fully immersed in performing an activity. You then experience a feeling of energized focus, full involvement, and enjoyment in the process of this activity',
     'DAILY_STEPS' : '[Between 0 and 10] HOW MANY STEPS (IN THOUSANDS) DO YOU TYPICALLY WALK EVERYDAY? Thousand steps, daily average over multiple days including work days and week-end.',
     'LIVE_VISION' : '[Between 0 and 10] FOR HOW MANY YEARS AHEAD IS YOUR LIFE VISION VERY CLEAR FOR? For instance, illustrated in a vision board, detailed in a personal journal or openly discussed with your spouse or close friends.',
     'SLEEP_HOURS' : '[Between 0 and 10] ABOUT HOW LONG DO YOU TYPICALLY SLEEP? Over the course of a typical working week, including week-end.',
     'LOST_VACATION' : '[Between 0 and 10] HOW MANY DAYS OF VACATION DO YOU TYPICALLY LOSE EVERY YEAR ? Unused vacation days, lost or carried forward into the following year. Or because of work stress during your vacation.',
     'DAILY_SHOUTING' : '[Between 0 and 10] HOW OFTEN DO YOU SHOUT OR SULK AT SOMEBODY? In a typical week. Expressing your negative emotions in an active or passive manner.',
     'SUFFICIENT_INCOME' : '[1 for insufficient, 2 for sufficient] HOW SUFFICIENT IS YOUR INCOME TO COVER BASIC LIFE EXPENSES? Such as the costs of housing, food, health care, car and education.',
     'PERSONAL_AWARDS' : '[Between 0 and 10] HOW MANY RECOGNITIONS HAVE YOU RECEIVED IN YOUR LIFE? Significant public recognitions validating a personal level of expertise and engagement E.g.: diploma, degree, certificate, accreditation, award, prize, published book, presentation at major conference, medals, cups, titles...',
     'TIME_FOR_PASSION' : '[Between 0 and 10] HOW MANY HOURS DO YOU SPEND EVERYDAY DOING WHAT YOU ARE PASSIONATE ABOUT? Daily hours spent doing what you are passionate and dreaming about, and/or contributing to a greater cause: health, education, peace, society development, ...',
     'DAILY_MEDITATION' : '[Between 0 and 10] IN A TYPICAL WEEK, HOW MANY TIMES DO YOU HAVE THE OPPORTUNITY TO THINK ABOUT YOURSELF? Include meditation, praying and relaxation activities such as fitness, walking in a park or lunch breaks.',
     'AGE' : "[1 = 'Less than 20' 2 = '21 to 35' 3 = '36 to 50' 4 = '51 or more']",
     'GENDER' : "[1 = 'Female' 0 = 'Male']"
 }

# Stats relating to each feature from the dataset, the first value is the mean second is the standard deviation. Used for feature scaling 
wellbeing_stats = {
     'FRUITS_VEGGIES': (2.9302234417875344, 1.4520159201380387),
     'DAILY_STRESS': (2.782673461387691, 1.3727325905540064),
     'PLACES_VISITED': (5.3392395139161115, 3.3139418123797992),
     'CORE_CIRCLE': (5.484907879263034, 2.8515330559044756),
     'SUPPORTING_OTHERS': (5.577420619364955, 3.2623788346607676),
     'SOCIAL_NETWORK': (6.551156409251274, 3.0838901957513056),
     'ACHIEVEMENT': (3.9632301058408466, 2.7521322641444677),
     'DONATION': (2.700744805958448, 1.8638952781722418),
     'BMI_RANGE': (1.4007056056448453, 0.49006066145045324),
     'TODO_COMPLETED': (5.706232849862799, 2.6333683190570336),
     'FLOW': (3.1263818110544883, 2.3345734226513537),
     'DAILY_STEPS': (5.704821638573109, 2.885451677609427),
     'LIVE_VISION': (3.71203449627597, 3.2444123653968657),
     'SLEEP_HOURS': (7.035201881615053, 1.2022010273941999),
     'LOST_VACATION': (2.8325362602900825, 3.671601401158279),
     'DAILY_SHOUTING': (2.920658565268522, 2.6898030587003587),
     'SUFFICIENT_INCOME': (1.7281850254802038, 0.44491247827457986),
     'PERSONAL_AWARDS': (5.702626421011368, 3.108526039697426),
     'TIME_FOR_PASSION': (3.266483731869855, 2.7212108297140687),
     'DAILY_MEDITATION': (6.253155625245002, 3.0348656183550693),
     'AGE': (2.581967855742846, 0.9537301053814297),
     'GENDER': (0.6047824382595061, 0.48891653834763144)
}

In [3]:
# Here we load the model, trained in a different notebook
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model('../input/schorsismodelterrarium/wellbeing_model')

In [4]:
### This cell is to test that the model is working properly, comment out the code below if you wish to run it


# wellbeing = pd.read_csv('../input/lifestyle-and-wellbeing-data/Wellbeing_and_lifestyle_data.csv')
# wellbeing = wellbeing.drop('Timestamp', axis=1)
# wellbeing = wellbeing.drop([10005])
# age_dict = {'Less than 20' : 1, '21 to 35' : 2, '36 to 50' : 3, '51 or more' : 4}
# wellbeing['AGE'] = pd.Series([age_dict[x] for x in wellbeing.AGE], index=wellbeing.index)
# gender_dict = {'Female' : 1, 'Male' : 0}
# wellbeing['GENDER'] = pd.Series([gender_dict[x] for x in wellbeing.GENDER], index=wellbeing.index)
# wellbeing['DAILY_STRESS'] = wellbeing['DAILY_STRESS'].astype(int)
# X = wellbeing.drop(['DAILY_STRESS', 'ACHIEVEMENT', 'BMI_RANGE', 'SUFFICIENT_INCOME', 'GENDER'], axis=1)
# for col in X.columns:
#     X[col] = (X[col]-X[col].mean())/X[col].std()
# y = wellbeing[['DAILY_STRESS', 'ACHIEVEMENT']]
# preds = model.predict(X)
# from sklearn import metrics
# print('Mean squared error: ', metrics.mean_squared_error(preds, y))

In [5]:
list_pred = []
try:
    for col in x_columns:
        print(wellbeing_dict[col])
        list_pred.append(int(input()))
except: # will execute when commiting the notebook or if user enters an invalid input
    list_pred = [3 for col in x_columns]

[Between 1 and 5] HOW MANY FRUITS OR VEGETABLES DO YOU EAT EVERYDAY? In a typical day, averaging workdays and weekends.


In [6]:
# Here the answer is transformed in to a batch of answers, one row the original answers and many more that are very similar
pred_dict = {}
pred_dict['Actual'] = list_pred
idx = 0
for num in list_pred:
    if num < 10:
        plus_1 = list_pred.copy()
        plus_1[idx] = num + 1
        label = str(x_columns[idx]) + ' plus 1'
        pred_dict[label] = plus_1
    if num > 0:
        minus_1 = list_pred.copy()
        minus_1[idx] = num - 1
        label = str(x_columns[idx]) + ' minus 1'
        pred_dict[label] = minus_1
    idx+=1
personal_preds_batch = pd.DataFrame.from_dict(pred_dict, orient='index', columns = x_columns)

# Here the data is scaled before its fed to the model
for col in x_columns:
    personal_preds_batch[col] = (personal_preds_batch[col]-wellbeing_stats[col][0])/wellbeing_stats[col][1]

Below are the predictions based on the input. The first row is the results based on the actual input, after that, it has estimates for if a user made a change. The row names describe the change to have the alternate outcome. A color gradient has been applied to help more significant changes stick out better. 

In [7]:
personal_preds_batch[['DAILY_STRESS', 'ACHIEVEMENT']] = model.predict(personal_preds_batch)
personal_preds_batch[['DAILY_STRESS', 'ACHIEVEMENT']].style.background_gradient()

,DAILY_STRESS,ACHIEVEMENT
Actual,3.057860,2.953829
FRUITS_VEGGIES plus 1,3.179225,2.735515
FRUITS_VEGGIES minus 1,2.933717,3.033097
PLACES_VISITED plus 1,3.062891,3.059111
PLACES_VISITED minus 1,3.066499,2.802756
CORE_CIRCLE plus 1,3.017951,2.990203
CORE_CIRCLE minus 1,3.098454,2.887364
SUPPORTING_OTHERS plus 1,3.156942,2.951120
SUPPORTING_OTHERS minus 1,2.956501,2.890684
SOCIAL_NETWORK plus 1,3.080816,2.960638


### Thanks for your time!
If you tried this notebook out and tried applying the recommendations let me know in the comments how it works out for you. Feel free to leave a comment with any issues, questions, or concerns. I have some future plans to expand this project and user feedback is important to me.